#Comment réduire la durée du traitement
>Il existe plusieurs techniques et méthodes pour réduire la durée du traitement.
Voici les principales:
- Utiliser une librairie de traitement de données qui est optimisée pour le traitement parallèle (pandas vs pyspark).
- Nettoyer le jeux de données (data cleaning).
- Choisir un format de fichier plus perfomant (.csv vs .parquet)
- Utiliser un cluster plus puissant ou mieux adapté au type de calcul.

## Utiliser la librairie pyspark ou pyspark.pandas
>Les librairie pyspark et pyspark.pandas permettent le traitement sur plusieurs ordinateurs:
- Ceci permet, entre autre, déviter les dépassements en mémoire, dans le cas ou le fichier a traiter serait très gros.
- Accèlere le traitement car plusieurs ordinateur vont traiter en même temps chacun une partie distincte du fichier.
***
[NOTE] Pour des perfomance optimal, ne pas oublier de:
- Désactiver le mode ANSI pour PySpark.Pandas
- Activer le mode ARROW
voir exemple ci-dessous

In [0]:
from pyspark.sql import SparkSession #utilisez pour configurer certaines optimisations
import pyspark.pandas as pd #Bon car le traitement va se faire en parallele sur plusieurs ordinateur
import pandas #Moins bon car le traitement va se faire sur un seul ordinateur

#Optimisation
if spark is None:
    spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.ansi.enabled", "False")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

### Élément important à considérer lors de la conversion de Pandas vers PySpark.Pandas.
>Il se peut que quelque fonctionalitées de Pandas ne soit pas disponible pour Pyspark.Pandas. Ceci est normal car Pandas à été designé pour fonctionner sur un seul ordinateur alors que PySpark.Pandas a été designer pour fonctionner de facon distribué sur plusieurs ordinateurs. En conséquence, les DataFrames de Pandas sont mutable (peuvent être modifié) et ceux de PySpark/PySpark.Pandas sont immutable (ne peuvent pas être modifié).

ref: https://spark.apache.org/docs/latest/api/python/tutorial/pandas_on_spark/supported_pandas_api.html

Voir les exemples ci-dessous

####Modifier un DataFrame Pandas partie 1
>Le DataFrame de Pandas est mutable, ce qui lui permet de faire ses modifications directement sur le DataFrame.

Voir exemple ci-dessous

In [0]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df.drop('A', axis=1, inplace=True) #modification directement fait dans le dataframe

#### Modifier un DataFrame Pandas partie 2
>Pandas, peut aussi renvoyer un nouveau DataFrame pour ses transformations, Pandas est généralement utilisé de cette façon, pour les raisons suivantes:
- Permet de conserver et réutilisez le DataFrame d'origine.
- Ajout de la clareté au code.

Voir exemple ci-dessous

In [0]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = df.drop('A', axis=1) #modification fait en renvoyant un nouveau dataframe

#### Modifier un DataFrame PySpark.Pandas
>Le DataFrame de PySpark.Pandas est immutable, donc seul les modifications fait en renvoyant un nouveau dataframe sont valide

Voir exemple ci-dessous

In [0]:
import pyspark.pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = df.drop('A', axis=1) #modification fait en renvoyant un nouveau dataframe

#### Executer PyPspark.Pandas avec le paramètre inplace=True
>Si vous tenter d'executer une cellule avec une modification inplace, pour PySpark.Pandas, vous risquer de recevoir ce message d'erreur: *TypeError: DataFrame.drop() got an unexpected keyword argument 'inplace'*

Voir exemple ci-dessous

In [0]:
import pyspark.pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df.drop('A', axis=1, inplace=True) #Erreur

#### Quoi faire si je dois utiliser une fonction qui existe seulement sur Pandas
>Il est quand même possible d'appelez une fonction qui existe seulement dans Pandas en utilisant la fonction de conversion to_pandas et from_pandas.
- to_pandas, rammenez les données dans un seul ordinateur ce qui rend le DataFrame mutable.
- from_pandas, distribut les données as travers plusieurs ordinateur, ce qui rend le dataframe immutable.
- Coût en perfomance élévé, donc as éviter

Voir exemple ci-dessous

In [0]:
import pandas as pd
import pyspark.pandas as pspd

#Chargement du DataFrame PySpark.Pandas (immutable)
psdf = pspd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

#Conversion de PySpark.Pandas en DataFrame Pandas (mutable)
df = psdf.to_pandas()
df.drop('A', axis=1, inplace=True)

#Reconvertir en PySpark.Pandas
psdf = pspd.from_pandas(df)

#Remarquer que la colone a bel et bien été retiré
display(psdf)

## Nettoyer le jeux de données
>Le but du nettoyage de données est d'éliminer les données qui ne seront pas utilisé dans le cadre du calcul actuariel. 
- Permet de diminuer le temps de chargement.
- Permet de réduire le temps des requêtes.
***
Pour l'exemple ci-dessous:
- Nous ramenons seulement les colones requisent pour le calcule (FMID,MarketName, Zip).

In [0]:
import pyspark.pandas as pd

path = "dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001/market_data.csv"
df = pd.read_csv(path,usecols=['FMID','MarketName','zip'])

print(f"Nombre de colones {df.columns.size}")
display(df.head())

## Choisir et favoriser le format de fichier .parquet
>Apache Parquet est un format de fichier en colonnes qui fournit des optimisations pour accélérer les requêtes. Il s'agit d'un format de fichier plus efficace que CSV ou JSON.
- Optimisé pour storer de large quantité de données. Les fichiers parquet sont au dessus de 10 fois plus léger que l'équivalent en .CSV ou .JSON
- Accélère massivement le temps de traitement en comparaison d'un fichier .CSV ou .JSON.
- Réduit considérablement les côuts de traitement.
***
Voici un tableau de comparaison entre .parquet et .CSV
| Type de fichier | Poids | Temps de réponse (query) | Qty de données scanné par la query |  Couts |
|-----------------|-------|--------------------------|------------------------------------|--------|
| .CSV | 1TB | 236 seconds | 1.15TB | 5.75$|
| .parquet | 130GB | 6.78 seconds | 2.51GB | 0.01$ |
| Gains | 87% plus petit | 34x rapide | 99% moins de traitement | 99.7% d'économie |

Comme le tableau le démontre, le format .parqet est massivement plus perfomant et économique. Il y a donc aucune raison logique d'utiliser un fichier .CSV ou JSON pour le traitement de vos données.

Nous allons vous montrer dans la cellule plus bas:
- Comment charger un fichier .parquet
- Comment convertir votre fichier .csv en fichier .parquet. 

La même logique s'applique as tout les autres formats de fichiers.

In [0]:
import pyspark.pandas as pd
from pyspark.sql import SparkSession

#Optimisation
if spark is None:
    spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.ansi.enabled", "False")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

#Variables
chemain_fichier_csv = "/Volumes/alpha_cloud_ai_workspace/default/exp-data/hour.csv"
destination_fichier_parquet = "/Volumes/alpha_cloud_ai_workspace/default/exp-data/hour.parquet"

#Lecture du CSV et Conversion en .parquet
df = pd.read_csv(chemain_fichier_csv)
df.to_parquet(destination_fichier_parquet)

#Rechargement du fichier en parquet
df = pd.read_parquet(destination_fichier_parquet)

#Faire votre traitement & Calcul
display(df.head(10))